In [2]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
#import time_series_functions as tsf
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


In [3]:
# Import data from csv file
df = pd.read_csv('/Users/jeanstibel/Documents/AI Classes/Projects/02 Project/Project 02/Resources/df_top_10.csv')
df.head()

,date,crypto_id,open,close,high,low,volume,number_of_trades,diff_oc,return,volatility,vol_change,ma_5,ma_10,price_movement
0,2020-09-22,AVAX,0.8500,5.3193,7.0000,0.8500,31441883.16,375884,4.4693,525.800000,723.529412,0.000000,0.00000,0.0,1
1,2020-09-23,AVAX,5.3279,3.5350,5.3600,3.4000,12620608.01,168336,-1.7929,-33.651157,36.787477,-59.860521,0.00000,0.0,0
2,2020-09-24,AVAX,3.5305,4.6411,4.8873,3.4033,7107843.86,104806,1.1106,31.457301,42.033706,-43.680654,0.00000,0.0,1
3,2020-09-25,AVAX,4.6366,4.7134,5.0234,4.2001,3886096.71,78381,0.0768,1.656386,17.756546,-45.326645,0.00000,0.0,1
4,2020-09-26,AVAX,4.7164,4.5200,4.8200,4.3294,1814932.97,34016,-0.1964,-4.164193,10.402002,-53.296763,4.54576,0.0,0


In [4]:
# Get list of'crypto_id's
crypto_ids = df['crypto_id'].unique()
crypto_ids

array(['AVAX', 'BTC', 'CRV', 'DEXE', 'DOGE', 'ENA', 'ENJ', 'FIL', 'GRT',
       'ICP', 'IOTA', 'MANA', 'QTUM', 'SAND', 'SHIB', 'SOL', 'TFUEL',
       'UNI'], dtype=object)

In statistics, the standard deviation is a measure of the amount of variation of the values of a variable about its mean. A low standard deviation indicates that the values tend to be close to the mean of the set, while a high standard deviation indicates that the values are spread out over a wider range.

In [5]:
# Calculate the standard deviation of the 'close' prices for each 'crypto_id'
crypto_volatility = df.groupby('crypto_id')['volatility'].std().reset_index()

# get volatility in order ascending
crypto_volatility = crypto_volatility.sort_values(by='volatility', ascending=True)

# get the 10 lest volatile crypto
crypto_volatility = crypto_volatility.head(10)

crypto_volatility

,crypto_id,volatility
1,BTC,3.967260
10,IOTA,6.093208
12,QTUM,6.271547
15,SOL,6.688596
6,ENJ,7.229954
16,TFUEL,9.290398
11,MANA,10.042802
14,SHIB,10.472944
2,CRV,11.629367
5,ENA,12.743082


In [6]:
def prepare_data_for_prophet(df, crypto_id):
    # Filter the data for the specified cryptocurrency
    df_crypto = df[df['crypto_id'] == crypto_id]

    # Prepare the data for Prophet
    df_crypto = df_crypto[['date', 'volatility']]
    df_crypto.columns = ['ds', 'y']

    # Convert the 'ds' column to datetime
    df_crypto['ds'] = pd.to_datetime(df_crypto['ds'])

    # Aggregate data to daily level (if necessary)
    df_crypto = df_crypto.groupby('ds').mean().reset_index()

    return df_crypto

# Prepare data for the top 10 least volatile crypto_ids
top_10_low_volatile_ids = crypto_volatility['crypto_id'].unique()
prepared_data_top_10_low_volatile = {crypto_id:
    prepare_data_for_prophet(df, crypto_id) for crypto_id in top_10_low_volatile_ids}


In [7]:
# Loop through each dataframe in the dictionary and fit the model
for crypto_id, df_prepared in prepared_data_top_10_low_volatile.items():
    
	# Initialize the Prophet model
	model = Prophet()
	
	# Fit the model on the data
	model.fit(df_prepared)
	
	# Create a dataframe to hold predictions
	future = model.make_future_dataframe(periods=30)
	
	# Make predictions
	forecast = model.predict(future)
	
	# Print the forecast for the current cryptocurrency
	display(f'Forecast for {crypto_id}:')
	display(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

21:52:49 - cmdstanpy - INFO - Chain [1] start processing
21:52:49 - cmdstanpy - INFO - Chain [1] done processing


'Forecast for BTC:'

,ds,yhat,yhat_lower,yhat_upper
0,2017-08-17,10.560776,5.500805,14.654532
1,2017-08-18,10.335265,5.772437,14.764564
2,2017-08-19,8.511822,3.420278,13.226597
3,2017-08-20,8.980217,4.526449,13.489805
4,2017-08-21,10.354268,6.071852,14.474091


21:52:49 - cmdstanpy - INFO - Chain [1] start processing
21:52:50 - cmdstanpy - INFO - Chain [1] done processing


'Forecast for IOTA:'

...

'Forecast for CRV:'

,ds,yhat,yhat_lower,yhat_upper
0,2020-08-15,36.288220,22.828796,49.782248
1,2020-08-16,34.271047,21.475171,47.097143
2,2020-08-17,36.108889,23.833479,49.083639
3,2020-08-18,35.528294,22.198013,48.859952
4,2020-08-19,35.352780,22.119830,47.623280


21:52:51 - cmdstanpy - INFO - Chain [1] start processing
21:52:51 - cmdstanpy - INFO - Chain [1] done processing


'Forecast for ENA:'

,ds,yhat,yhat_lower,yhat_upper
0,2024-04-02,34.273726,19.702603,49.026785
1,2024-04-03,32.362769,17.292132,46.803448
2,2024-04-04,29.194720,14.999626,43.479834
3,2024-04-05,30.470192,16.305813,44.711373
4,2024-04-06,26.745514,10.375803,41.791211


In [8]:
# Create a dataframe to hold predictions
future = model.make_future_dataframe(periods=30)

# Make predictions
forecast = model.predict(future)

In [9]:
# Print the forecast for the current cryptocurrency and save the results in the file 'Forecast'

for crypto_id in top_10_low_volatile_ids:
    df_prepared = prepared_data_top_10_low_volatile[crypto_id]
    
    # Initialize the Prophet model
    model = Prophet()
    
    # Fit the model on the data
    model.fit(df_prepared)
    
    # Create a dataframe to hold predictions
    future = model.make_future_dataframe(periods=30)
    
    # Make predictions
    forecast = model.predict(future)
    
    # Plot the forecast
    fig1 = model.plot(forecast)
    plt.title(f'{crypto_id} Volatility Price Forecast')
    plt.xlabel('Date')
    plt.ylabel('Volatility Price')
    plt.savefig(f'/Users/jeanstibel/Documents/AI Classes/Projects/project_02/Forecast/{crypto_id}_forecast.png')
    plt.close(fig1)
    
    # Plot the forecast components
    fig2 = model.plot_components(forecast)
    plt.suptitle(f'{crypto_id} Forecast Components', fontsize=16)
    plt.savefig(f'/Users/jeanstibel/Documents/AI Classes/Projects/project_02/Forecast/{crypto_id}_forecast_components.png')
    plt.close(fig2)



21:52:51 - cmdstanpy - INFO - Chain [1] start processing
21:52:51 - cmdstanpy - INFO - Chain [1] done processing
21:52:52 - cmdstanpy - INFO - Chain [1] start processing
21:52:52 - cmdstanpy - INFO - Chain [1] done processing
21:52:52 - cmdstanpy - INFO - Chain [1] start processing
21:52:52 - cmdstanpy - INFO - Chain [1] done processing
21:52:53 - cmdstanpy - INFO - Chain [1] start processing
21:52:53 - cmdstanpy - INFO - Chain [1] done processing
21:52:53 - cmdstanpy - INFO - Chain [1] start processing
21:52:53 - cmdstanpy - INFO - Chain [1] done processing
21:52:53 - cmdstanpy - INFO - Chain [1] start processing
21:52:53 - cmdstanpy - INFO - Chain [1] done processing
21:52:54 - cmdstanpy - INFO - Chain [1] start processing
21:52:54 - cmdstanpy - INFO - Chain [1] done processing
21:52:54 - cmdstanpy - INFO - Chain [1] start processing
21:52:54 - cmdstanpy - INFO - Chain [1] done processing
21:52:54 - cmdstanpy - INFO - Chain [1] start processing
21:52:54 - cmdstanpy - INFO - Chain [1]

In [10]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE for each of the top 10 low volatile cryptocurrencies
for crypto_id in top_10_low_volatile_ids:
    df_prepared = prepared_data_top_10_low_volatile[crypto_id]
    
    # Ensure the lengths of the dataframes are the same
    min_length = min(len(df_prepared), len(forecast))
    df_prepared = df_prepared.iloc[:min_length]
    forecast_trimmed = forecast.iloc[:min_length]
    
    mae = mean_absolute_error(df_prepared['y'], forecast_trimmed['yhat'])
    print(f'Mean Absolute Error for {crypto_id}: {mae}')

Mean Absolute Error for BTC: 7.251466790400175
Mean Absolute Error for IOTA: 6.887395242898888
Mean Absolute Error for QTUM: 6.321945385292588
Mean Absolute Error for SOL: 6.169704230539761
Mean Absolute Error for ENJ: 6.292023338446587
Mean Absolute Error for TFUEL: 6.856254433020692
Mean Absolute Error for MANA: 7.0504115843508774
Mean Absolute Error for SHIB: 7.930981903032593
Mean Absolute Error for CRV: 8.342744254696001
Mean Absolute Error for ENA: 4.914553630511177
